weatherAUS 数据集 BT-TWD 可行性实验

本 notebook 按步骤运行：加载配置 → 读取数据 → 预处理 → 桶树划分 → 基线与 BTTWD k 折实验 → 桶级分析。

In [1]:
# 步骤0：环境与路径设置
import os, sys
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['Microsoft YaHei']
plt.rcParams['axes.unicode_minus'] = False

# 将项目根目录加入路径，便于导入 bttwdlib
root_path = os.path.abspath(os.path.join(os.getcwd(), '..'))
if root_path not in sys.path:
    sys.path.append(root_path)

from bttwdlib import (
    load_yaml_cfg,
    show_cfg,
    load_dataset,
    prepare_features_and_labels,
    BucketTree,
    run_kfold_experiments,
    log_info,
    set_global_seed,
)

cfg_path = Path(root_path) / "configs" / "credit_default_bttwd.yaml"
cfg = load_yaml_cfg(cfg_path)
set_global_seed(cfg.get('SEED', {}).get('global_seed', 42))
log_info('【步骤0摘要】环境准备完毕，路径与随机种子已设置。')

【INFO】【2025-12-16 15:43:46】【配置加载】已读取 e:\yan\组\三支决策\机器学习\BT_TWD\configs\credit_default_bttwd.yaml
【INFO】【2025-12-16 15:43:49】【步骤0摘要】环境准备完毕，路径与随机种子已设置。


In [2]:
# 步骤1：加载配置
show_cfg(cfg)
log_info('【步骤1摘要】配置文件加载完成，关键参数检查通过。')

【INFO】【2025-12-16 15:43:49】【配置-数据】数据集=credit_default, k折=None, 目标列=default payment next month, 正类="1"
【INFO】【2025-12-16 15:43:49】【配置-BTTWD】阈值模式=bucket_wise, 全局模型=xgb, 桶内模型=none, 后验估计器(兼容字段)=logreg
【INFO】【2025-12-16 15:43:49】【配置-基线】LogReg启用=True, RandomForest启用=True, KNN启用=True, XGBoost启用=True
【INFO】【2025-12-16 15:43:49】【步骤1摘要】配置文件加载完成，关键参数检查通过。


In [3]:
# 步骤2：加载原始数据
df_raw, target_col_model = load_dataset(cfg)  # 这里返回的是用于建模的标签列，例如 "label"

display(df_raw.head())
print("用于建模的标签列:", target_col_model)


# 1）画 0/1 标签（流失/未流失）的比例
class_counts = df_raw[target_col_model].value_counts(normalize=True)
ax = class_counts.plot(kind='bar', title='流失 vs 未流失比例')
plt.ylabel('比例')

fig_path = os.path.join(root_path, cfg['OUTPUT']['figs_dir'], 'class_distribution.png')
os.makedirs(os.path.dirname(fig_path), exist_ok=True)
plt.savefig(fig_path, bbox_inches='tight')
plt.close()

# 2）如果想看原始标签列的分布，可以另外单独分析：
raw_target_col = cfg['DATA']['target_col']  # 这里是原始标签列
print("原始目标列:", raw_target_col)
print(df_raw[raw_target_col].describe())

log_info('【步骤2摘要】Telco Churn 原始数据加载与基本统计完成。')


【INFO】【2025-12-16 15:43:51】【数据加载】目标列 default payment next month 已检测为 0/1 标签，跳过映射逻辑
【INFO】【2025-12-16 15:43:51】【数据集信息】名称=credit_default，样本数=30000，目标列=default payment next month，正类比例=22.12%


,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
1,2,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,1,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


用于建模的标签列: default payment next month
原始目标列: default payment next month
count    30000.000000
mean         0.221200
std          0.415062
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: default payment next month, dtype: float64
【INFO】【2025-12-16 15:43:51】【步骤2摘要】Telco Churn 原始数据加载与基本统计完成。


In [4]:
# 步骤3：预处理与特征工程
X, y, meta = prepare_features_and_labels(df_raw, cfg)
log_info(f'【预处理】编码特征维度={X.shape[1]}，样本数={X.shape[0]}')
log_info(f"【步骤3摘要】特征预处理完成：连续={len(meta['continuous_cols'])}，类别={len(meta['categorical_cols'])}，编码维度={X.shape[1]}。")

【INFO】【2025-12-16 15:43:51】【预处理】缺失值填充策略=median
【INFO】【2025-12-16 15:43:51】已生成 credit_default 派生特征：ever_delay / max_delay / max_delay_bin
【INFO】【2025-12-16 15:43:51】ever_delay 分布：
ever_delay
0    19931
1    10069
【INFO】【2025-12-16 15:43:51】max_delay_bin 分布：
max_delay_bin
0      19931
1-2     8876
3-4     1007
5+       186
【INFO】【2025-12-16 15:43:51】max_delay_bins=[-0.1, 0, 2, 4, 9], labels=['0', '1-2', '3-4', '5+']
【INFO】【2025-12-16 15:43:51】【预处理】连续特征=14个，类别特征=5个
【INFO】【2025-12-16 15:43:51】【预处理】编码后维度=33
【INFO】【2025-12-16 15:43:51】【预处理】编码特征维度=33，样本数=30000
【INFO】【2025-12-16 15:43:51】【步骤3摘要】特征预处理完成：连续=14，类别=5，编码维度=33。


In [5]:
# 步骤4：构建桶树并检查划分（✅与主流程一致：使用步骤3生成的 df_processed）
df_processed = meta.get("df_processed", df_raw)

# 只取桶层真正用到的列，避免把一堆无关列塞进分桶检查
bucket_cols = []
for lvl in cfg["BTTWD"]["bucket_levels"]:
    col_name = lvl.get("col") or lvl.get("feature")
    if col_name and col_name not in bucket_cols:
        bucket_cols.append(col_name)

bucket_tree = BucketTree(
    cfg["BTTWD"]["bucket_levels"],
    feature_names=bucket_cols
)

# ✅关键：用 df_processed（已包含 ever_delay / max_delay_bin）
bucket_ids_full = bucket_tree.assign_buckets(df_processed[bucket_cols])

bucket_df = bucket_ids_full.value_counts().reset_index()
bucket_df.columns = ["bucket_id", "count"]

# 正类比例也用 df_processed 对齐（目标列仍在 df_processed 里）
bucket_df["pos_rate"] = (
    df_processed.groupby(bucket_ids_full)[target_col_model]
    .apply(lambda s: (s == 1).mean())
    .values
)

display(bucket_df.head())

bucket_df.set_index("bucket_id")["count"].plot(
    kind="bar", figsize=(12, 4), title="桶样本数分布"
)
fig_bucket = os.path.join(root_path, cfg["OUTPUT"]["figs_dir"], "bucket_metrics_bar.png")
plt.savefig(fig_bucket, bbox_inches="tight")
plt.close()

log_info(f"【步骤4摘要】桶树划分完成，共有 {bucket_ids_full.nunique()} 个叶子桶。")


【INFO】【2025-12-16 15:43:51】【桶树】已为样本生成桶ID，共 12 个组合


,bucket_id,count,pos_rate
0,L1_ever_delay=no_delay|L2_max_delay_bin=none|L...,7565,0.697674
1,L1_ever_delay=no_delay|L2_max_delay_bin=none|L...,6468,0.612500
2,L1_ever_delay=no_delay|L2_max_delay_bin=none|L...,5898,0.646766
3,L1_ever_delay=has_delay|L2_max_delay_bin=mild|...,4044,0.317972
4,L1_ever_delay=has_delay|L2_max_delay_bin=mild|...,2879,0.447577


【INFO】【2025-12-16 15:43:52】【步骤4摘要】桶树划分完成，共有 12 个叶子桶。


In [6]:
# 步骤5：运行基线模型 k 折实验
# 基线部分在 run_kfold_experiments 内统一调度
log_info('【步骤5】基线模型将在整体交叉验证中一并运行。')
log_info('【步骤5摘要】基线模型性能将作为后续对比基准。')

【INFO】【2025-12-16 15:43:52】【步骤5】基线模型将在整体交叉验证中一并运行。
【INFO】【2025-12-16 15:43:52】【步骤5摘要】基线模型性能将作为后续对比基准。


In [7]:
import numpy as np
import pandas as pd

print("y 全局标签分布：", np.unique(y, return_counts=True))

print("原始标签列分布：")
print(df_raw[cfg['DATA']['target_col']].value_counts())


y 全局标签分布： (array([0, 1]), array([23364,  6636], dtype=int64))
原始标签列分布：
default payment next month
0    23364
1     6636
Name: count, dtype: int64


In [8]:
# 步骤6：运行 BTTWD k 折实验（含基线）——✅使用 df_processed 作为分桶输入
df_processed = meta.get("df_processed", df_raw)

# 训练/分桶用的 df 不能包含标签列
target_col = cfg["DATA"]["target_col"]
X_df_for_bucket = df_processed.drop(columns=[target_col], errors="ignore")

results = run_kfold_experiments(X, y, X_df_for_bucket, cfg)

summary_path = os.path.join(root_path, cfg["OUTPUT"]["results_dir"], "metrics_kfold_summary.csv")
summary_df = pd.read_csv(summary_path)
display(summary_df)


【INFO】【2025-12-16 15:43:52】【基线-LogReg】使用决策阈值=0.300（fixed 模式）


e:\yan\组\三支决策\机器学习\BT_TWD\bttwdlib\baselines.py:103: RuntimeWarning: Mean of empty slice
  summary[f"{col}_mean"] = float(np.nanmean(arr))
d:\Anaconda3\Lib\site-packages\numpy\lib\nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


【INFO】【2025-12-16 15:43:53】【基线-LogReg】整体指标：AUC_mean=0.745, AUC_std=0.006, BAC_mean=0.702, BAC_std=0.005, BND_ratio_mean=0.000, BND_ratio_std=0.000, F1_mean=0.520, F1_std=0.006, Kappa_mean=0.353, Kappa_std=0.009, MCC_mean=0.363, MCC_std=0.009, POS_Coverage_mean=nan, POS_Coverage_std=nan, Precision_mean=0.445, Precision_std=0.007, Recall_mean=0.627, Recall_std=0.014, Regret_mean=0.421, Regret_std=0.007
【INFO】【2025-12-16 15:43:53】【基线-RF】使用决策阈值=0.300（fixed 模式）


e:\yan\组\三支决策\机器学习\BT_TWD\bttwdlib\baselines.py:103: RuntimeWarning: Mean of empty slice
  summary[f"{col}_mean"] = float(np.nanmean(arr))
d:\Anaconda3\Lib\site-packages\numpy\lib\nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


【INFO】【2025-12-16 15:44:06】【基线-RF】整体指标：AUC_mean=0.755, AUC_std=0.004, BAC_mean=0.698, BAC_std=0.007, BND_ratio_mean=0.000, BND_ratio_std=0.000, F1_mean=0.517, F1_std=0.009, Kappa_mean=0.355, Kappa_std=0.011, MCC_mean=0.361, MCC_std=0.012, POS_Coverage_mean=nan, POS_Coverage_std=nan, Precision_mean=0.455, Precision_std=0.008, Recall_mean=0.599, Recall_std=0.014, Regret_mean=0.425, Regret_std=0.009
【INFO】【2025-12-16 15:44:06】【基线-KNN】使用决策阈值=0.300（fixed 模式）


...

e:\yan\组\三支决策\机器学习\BT_TWD\bttwdlib\bucket_gain.py:27: RuntimeWarning: compute_bucket_score: metric 'F1' is missing or NaN, defaulting to 0.0.
  warnings.warn(
e:\yan\组\三支决策\机器学习\BT_TWD\bttwdlib\bucket_gain.py:27: RuntimeWarning: compute_bucket_score: metric 'BND_ratio' is missing or NaN, defaulting to 0.0.
  warnings.warn(


【INFO】【2025-12-16 15:44:34】【阈值】桶 L1_ever_delay=has_delay（n_val=197）使用本地阈值 α=0.5000, β=0.0000
[INFO][BT][2025-12-16 15:44:34] 桶 bucket_id=L1_ever_delay=has_delay level=1：
    n_train=123, n_val=197,
    BAC=0.443, F1=0.768, AUC=0.661,
    Regret=0.373, BND_ratio=0.183, POS_coverage=0.817,
    Score(f1_regret_bnd )=0.303
[INFO][BT][2025-12-16 15:44:34] 桶 bucket_id=L1_ever_delay=has_delay：
    parent_id=ROOT，parent_Score=-0.385, bucket_Score=0.303,
    Gain=+0.688, is_weak=False
【INFO】【2025-12-16 15:44:34】【阈值】桶 L1_ever_delay=no_delay 标记为弱桶，阈值将回退使用 ROOT 的阈值
[INFO][BT][2025-12-16 15:44:34] 桶 bucket_id=L1_ever_delay=no_delay level=1：
    n_train=0, n_val=0,
    BAC=nan, F1=nan, AUC=nan,
    Regret=nan, BND_ratio=nan, POS_coverage=nan,
    Score(f1_regret_bnd )=nan
[INFO][BT][2025-12-16 15:44:34] 桶 bucket_id=L1_ever_delay=no_delay：
    parent_id=ROOT，parent_Score=-0.385, bucket_Score=nan,
    Gain=+nan, is_weak=True
【INFO】【2025-12-16 15:44:35】【阈值】桶 L1_ever_delay=has_delay|L2_max_delay_bin=med

C:\Users\xzq20.HUANLI\AppData\Roaming\Python\Python311\site-packages\xgboost\core.py:158: UserWarning: [15:44:38] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


【INFO】【2025-12-16 15:44:38】[BASELINE] 全局 XGB 模型训练完成
【INFO】【2025-12-16 15:44:38】[BASELINE] 阈值搜索开始
【INFO】【2025-12-16 15:44:38】[BASELINE] 最佳阈值找到: alpha=0.4000, beta=0.2000, regret=0.3974
【INFO】【2025-12-16 15:44:38】【桶树】已为样本生成桶ID，共 12 个组合
【INFO】【2025-12-16 15:44:38】[BASELINE] 测试集桶映射完成，共 12 个桶
【INFO】【2025-12-16 15:44:39】[BASELINE] 桶 L1_ever_delay=no_delay|L2_max_delay_bin=none|L3_LIMIT_BAL=low_limit: BAC=0.5072, Regret=0.4365, Precision=0.6000, Recall=0.0163
【INFO】【2025-12-16 15:44:39】[BASELINE] 桶 L1_ever_delay=no_delay|L2_max_delay_bin=none|L3_LIMIT_BAL=mid_limit: BAC=0.5050, Regret=0.3342, Precision=0.3333, Recall=0.0134
【INFO】【2025-12-16 15:44:39】[BASELINE] 桶 L1_ever_delay=no_delay|L2_max_delay_bin=none|L3_LIMIT_BAL=high_limit: BAC=0.5018, Regret=0.3047, Precision=0.2500, Recall=0.0058
【INFO】【2025-12-16 15:44:39】[BASELINE] 桶 L1_ever_delay=has_delay|L2_max_delay_bin=mild|L3_LIMIT_BAL=mid_limit: BAC=0.5936, Regret=0.5629, Precision=0.4385, Recall=0.6129
【INFO】【2025-12-16 15:44:39】[BASELINE]

,model,Precision_mean,Precision_std,Recall_mean,Recall_std,F1_mean,F1_std,BAC_mean,BAC_std,AUC_mean,...,MCC_mean,MCC_std,Kappa_mean,Kappa_std,BND_ratio_mean,BND_ratio_std,POS_Coverage_mean,POS_Coverage_std,Regret_mean,Regret_std
0,BTTWD,0.440813,0.010147,0.636826,0.013936,0.520967,0.011030,0.703664,0.008571,0.759398,...,0.362578,0.015455,0.351395,0.015195,0.153733,0.007913,0.231267,0.018195,0.404258,0.007903
1,LogReg,0.445227,0.007344,0.626579,0.013601,0.520454,0.006285,0.702350,0.005043,0.745270,...,0.362823,0.008697,0.353172,0.008633,0.000000,0.000000,NaN,NaN,0.420600,0.006671
2,RandomForest,0.455229,0.007578,0.598552,0.014403,0.517083,0.008805,0.697538,0.006730,0.755216,...,0.361081,0.011573,0.355029,0.011198,0.000000,0.000000,NaN,NaN,0.424867,0.008967
3,KNN,0.362512,0.006880,0.613016,0.015880,0.455562,0.009012,0.653409,0.007814,0.689636,...,0.263198,0.013130,0.245955,0.012017,0.000000,0.000000,NaN,NaN,0.495267,0.010505
4,XGBoost,0.470649,0.009512,0.589358,0.012818,0.523309,0.009693,0.700517,0.007042,0.762483,...,0.371966,0.012989,0.367809,0.012868,0.000000,0.000000,NaN,NaN,0.419167,0.009592


In [9]:
# 步骤7：桶级别分析
bucket_metrics_path = os.path.join(root_path, cfg['OUTPUT']['results_dir'], 'bucket_metrics.csv')
if os.path.exists(bucket_metrics_path):
    bucket_metrics_df = pd.read_csv(bucket_metrics_path)
    display(bucket_metrics_df.head())
    bucket_metrics_df.plot(x='bucket_id', y='pos_rate_all', kind='bar', figsize=(12,4), title='桶正类比例')
    plt.ylabel('正类比例')
    plt.xticks(rotation=90)
    plt.tight_layout()
    plt.savefig(fig_bucket, bbox_inches='tight')
    plt.close()
log_info('【步骤7摘要】桶级指标已整理，可用于局部化分析。')

,bucket_id,layer,parent_bucket_id,n_train,n_val,pos_rate_train,pos_rate_val,alpha,beta,regret_val,...,is_weak,threshold_source_bucket,parent_with_threshold,n_test,pos_rate_test,BND_ratio_test,POS_Coverage_test,regret_test,fold,pos_rate
0,ROOT,L1,NaN,0,0,NaN,NaN,0.4,0.2,NaN,...,True,ROOT,NaN,NaN,NaN,NaN,NaN,NaN,1,0.221167
1,L1_ever_delay=no_delay,L1,ROOT,0,0,NaN,NaN,0.4,0.2,NaN,...,True,ROOT,NaN,NaN,NaN,NaN,NaN,NaN,1,0.118268
2,L1_ever_delay=no_delay|L2_max_delay_bin=none,L2,L1_ever_delay=no_delay,0,0,NaN,NaN,0.4,0.2,NaN,...,True,ROOT,NaN,NaN,NaN,NaN,NaN,NaN,1,0.118268
3,L1_ever_delay=has_delay,L1,ROOT,109,195,0.715596,0.610256,0.4,0.2,0.380769,...,False,L1_ever_delay=has_delay,NaN,NaN,NaN,NaN,NaN,NaN,1,0.427195
4,L1_ever_delay=has_delay|L2_max_delay_bin=mild,L2,L1_ever_delay=has_delay,0,0,NaN,NaN,0.4,0.2,NaN,...,True,L1_ever_delay=has_delay,L1_ever_delay=has_delay,NaN,NaN,NaN,NaN,NaN,1,0.399886


C:\Users\xzq20.HUANLI\AppData\Local\Temp\ipykernel_26432\946468196.py:9: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all Axes decorations.
  plt.tight_layout()


【INFO】【2025-12-16 15:44:40】【步骤7摘要】桶级指标已整理，可用于局部化分析。


In [10]:
# 步骤8：结果汇总
log_info('【步骤8】检查结果文件与图表。')
print(os.listdir(os.path.join(root_path, cfg['OUTPUT']['results_dir'])))
print(os.listdir(os.path.join(root_path, cfg['OUTPUT']['figs_dir'])))
log_info('【全部步骤完成】Telco Churn 数据集上的 BT-TWD 实验结束。')

【INFO】【2025-12-16 15:44:40】【步骤8】检查结果文件与图表。
['baseline_bucket_metrics.csv', 'bucket_fallback_stats.csv', 'bucket_metrics.csv', 'bucket_metrics_gain.csv', 'bucket_thresholds.csv', 'bucket_thresholds_per_fold.csv', 'bucket_tree_structure.csv', 'metrics_kfold_per_fold.csv', 'metrics_kfold_summary.csv', 'metrics_overview.csv']
['bank_class_distribution.png', 'bucket_metrics_bar.png', 'class_distribution.png', 'metrics_compare.png']
【INFO】【2025-12-16 15:44:40】【全部步骤完成】Telco Churn 数据集上的 BT-TWD 实验结束。
